# Goal

To Web Scrape Zillow to see the average home price of the Dallas Area. ***Note that API is limited to pulling 500 Values***

### Links

Zillow: https://www.zillow.com/dallas-tx/

Zillow Sold: https://www.zillow.com/dallas-tx/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A33.041254272695596%2C%22south%22%3A32.594037410567665%2C%22east%22%3A-96.46741105322266%2C%22west%22%3A-97.08745194677735%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A38128%2C%22regionType%22%3A6%7D%5D%7D


### Imports

In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests

pd.set_option("display.max_columns", None)

### Functions

In [7]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

def convert_sqft_to_acres(dfs):
    # Conversion factor from square feet to acres
    sqft_to_acres = 1 / 43560

    # Using the .loc accessor to modify rows where the Lot Area Unit is 'sqft'
    # Convert and round the area to three decimal places
    dfs.loc[dfs['Lot Area Unit'] == 'sqft', 'Lot Area'] = (dfs['Lot Area'] * sqft_to_acres).round(3)
    dfs.loc[dfs['Lot Area Unit'] == 'sqft', 'Lot Area Unit'] = 'acres'
    
    return dfs


### Locals and Constants

In [8]:
api_key = "220f3c4c-6532-416a-99b2-8b3e85893f8e"

### Data

### 1. Property Listings for Sale

In [9]:
# zillow search url
listing_url = "https://www.zillow.com/dallas-tx/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A33.041254272695596%2C%22south%22%3A32.594037410567665%2C%22east%22%3A-96.46741105322266%2C%22west%22%3A-97.08745194677735%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A38128%2C%22regionType%22%3A6%7D%5D%7D"
# get listings
listing_response = get_listings(api_key, listing_url)

In [10]:
# view all keys
listing_response.json().keys()

# check if request is successful
listing_response.json()["is_success"]

# view count of properies returned in request
num_of_properties = listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

KeyError: 'categoryTotals'

In [ ]:
# view all listings
dfs_listings = pd.json_normalize(listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(dfs_listings))
print("Number of columns:", len(dfs_listings.columns))
dfs_listings

In [ ]:
# Export DataFrame to CSV file
dfs_listings.to_csv('listings_raw_sale.csv', index=False)

### Clean Up Data Frame

In [ ]:
dfs = pd.read_csv("listings_raw_sale.csv")

# Clean up column names
dfs.columns = dfs.columns.str.lower()  # Convert to lowercase
dfs.columns = dfs.columns.str.replace(' ', '_')  # Replace spaces with underscores
dfs.columns = dfs.columns.str.replace('[^a-zA-Z0-9_]', '')  # Remove special characters

# Display the list of column names
print("Column names:")
print(dfs.columns.tolist())

This code will:

Convert all column names to lowercase.
Replace spaces with underscores.
Remove any special characters from column names.

In [ ]:
# Keep specific columns
columns_to_keep_s = [
    'detailurl', 'statustext', 'area',

    'hdpdata.homeinfo.zpid', 'hdpdata.homeinfo.streetaddress', 'hdpdata.homeinfo.zipcode', 'hdpdata.homeinfo.city', 'hdpdata.homeinfo.state', 
    'hdpdata.homeinfo.latitude', 'hdpdata.homeinfo.longitude', 'hdpdata.homeinfo.price', 'hdpdata.homeinfo.datepricechanged', 
    'hdpdata.homeinfo.bathrooms', 'hdpdata.homeinfo.bedrooms', 'hdpdata.homeinfo.livingarea', 'hdpdata.homeinfo.hometype', 
    'hdpdata.homeinfo.homestatus', 'hdpdata.homeinfo.daysonzillow', 

    'hdpdata.homeinfo.zestimate', 'hdpdata.homeinfo.rentzestimate',

    'hdpdata.homeinfo.pricechange',

    'hdpdata.homeinfo.taxassessedvalue', 'hdpdata.homeinfo.lotareavalue', 'hdpdata.homeinfo.lotareaunit',
    ]
dfs[columns_to_keep_s]
# Select columns to keep
dfs = dfs[columns_to_keep_s]
dfs

In [ ]:
column_name_mapping_s = {
    'detailurl' : 'URL', 
    'statustext' : 'Status', 
    'hdpdata.homeinfo.zpid' : 'ZipID', 
    'area': "Sqft",
    'hdpdata.homeinfo.streetaddress' : 'StreetAddress', 
    'hdpdata.homeinfo.zipcode' : 'ZipCode', 
    'hdpdata.homeinfo.city' : 'City', 
    'hdpdata.homeinfo.state' : 'State', 
    'hdpdata.homeinfo.latitude' : 'Latitude', 
    'hdpdata.homeinfo.longitude' : 'Longitude', 
    'hdpdata.homeinfo.price' : 'Price', 
    'hdpdata.homeinfo.datepricechanged' : 'DatePriceChanged', 
    'hdpdata.homeinfo.bathrooms' : 'Baths', 
    'hdpdata.homeinfo.bedrooms' : 'Beds', 
    'hdpdata.homeinfo.livingarea' : 'Living Area', 
    'hdpdata.homeinfo.hometype' : 'Home Type', 
    'hdpdata.homeinfo.homestatus' : 'Home Status', 
    'hdpdata.homeinfo.daysonzillow' : 'Days on Zillow', 

    'hdpdata.homeinfo.zestimate' : 'Zestimate', 
    'hdpdata.homeinfo.rentzestimate' : 'RentZestimate',

    'hdpdata.homeinfo.pricechange' : 'PriceChange',

    'hdpdata.homeinfo.taxassessedvalue' : 'TaxAssessedValue', 
    'hdpdata.homeinfo.lotareavalue' : 'LotArea', 
    'hdpdata.homeinfo.lotareaunit' : 'LotAreaUnit',
}

# Rename columns
dfs.rename(columns=column_name_mapping_s, inplace=True)

In [ ]:
# Assuming dfs is your DataFrame
number_of_rows, number_of_columns = dfs.shape

print("Number of rows:", number_of_rows)
print("Number of columns:", number_of_columns)

dfs

In [ ]:
# Convert Lot Values from Sqft to Acres
# Call the function to convert areas
dfs = convert_sqft_to_acres(dfs)

In [ ]:
# Export DataFrame to CSV file
dfs.to_csv('listings_cleaned_sale.csv', index=False)